In [1]:
import pandas as pd
import sqlite3
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopy

In [2]:
#lists tables and indexes in the database
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM sqlite_master; "

master_sqlite = pd.read_sql(query, db)

db.close()

master_sqlite

,type,name,tbl_name,rootpage,sql
0,table,hop_team,hop_team,2,"CREATE TABLE ""hop_team"" (\n""from_npi"" INTEGER,..."
1,table,cbsa,cbsa,2876549,"CREATE TABLE ""cbsa"" (\n""zip"" INTEGER,\n ""cbsa..."
2,table,taxonomy,taxonomy,2877873,"CREATE TABLE ""taxonomy"" (\n""NPI"" INTEGER,\n ""..."
3,table,nucc,nucc,3298774,"CREATE TABLE ""nucc"" (\n""code"" TEXT,\n ""groupi..."
4,index,zip,cbsa,3298917,CREATE INDEX zip ON cbsa(zip)
5,index,NPI,taxonomy,3299319,CREATE INDEX NPI ON taxonomy(NPI)
6,index,code,nucc,3666969,CREATE INDEX code ON nucc(code)
7,table,nppes,nppes,3666974,"CREATE TABLE ""nppes"" (\n""npi"" INTEGER,\n ""ent..."
8,index,nppes_npi,nppes,3810771,CREATE INDEX nppes_npi ON nppes(npi)
9,index,from_npi,hop_team,2131546,CREATE INDEX from_npi ON hop_team(from_npi)


In [3]:
#Performs a SQL query of the hop_team database and pulls hospital systems in the Nashville CBSA that are being
#referred to from outside of the Nashville CBSA
#filters for entity_type_code = 2.0 and taxonomy_switch = 'Y'
query = '''
WITH to_npi AS(
SELECT npi AS to_npi,
    "provider_organization_name_(legal_business_name)" AS to_org_name,
    "provider_last_name_(legal_name)" AS to_last_name,
    provider_first_name AS to_first_name,
    provider_middle_name AS to_middle_name,
    provider_credential_text AS to_credential,
    provider_first_line_business_practice_location_address AS to_address,
    provider_second_line_business_practice_location_address AS to_2_address,
    provider_business_practice_location_address_city_name AS to_city,
    provider_business_practice_location_address_state_name AS to_state,
    substring(nppes.provider_business_practice_location_address_postal_code, 1, 5) AS to_zip
FROM nppes
WHERE entity_type_code = 2.0
),
Nash_cbsa AS(
    SELECT CAST(zip as TEXT) as zip,
        cbsa,
        usps_zip_pref_city,
        usps_zip_pref_state
    FROM cbsa 
    WHERE cbsa = 34980
),
taxon AS (
    SELECT npi, taxonomy_code
    FROM taxonomy
    WHERE taxonomy_switch = 'Y'
), 
codes AS(
    SELECT code, classification, specialization, display_name
    FROM nucc
)
SELECT *
FROM to_npi
JOIN Nash_cbsa
on to_npi.to_zip = zip
JOIN taxon
ON to_npi.to_npi = npi
JOIN codes
ON codes.code = taxon.taxonomy_code

'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    to_npi = pd.read_sql(query,db)

db.close()
to_npi.to_csv('data/to_npi.csv')
to_npi.head()

,to_npi,to_org_name,to_last_name,to_first_name,to_middle_name,to_credential,to_address,to_2_address,to_city,to_state,...,res_ratio,bus_ratio,oth_ratio,tot_ratio,npi,taxonomy_code,code,classification,specialization,display_name
0,1881697092,"RHS, INC.",None,None,None,None,1330 TROTWOOD AVE,None,COLUMBIA,TN,...,0.993028,0.999619,1.0,0.993669,1881697092,332B00000X,332B00000X,Durable Medical Equipment & Medical Supplies,None,Durable Medical Equipment & Medical Supplies
1,1881697092,"RHS, INC.",None,None,None,None,1330 TROTWOOD AVE,None,COLUMBIA,TN,...,0.993028,0.999619,1.0,0.993669,1881697092,332B00000X,332B00000X,Durable Medical Equipment & Medical Supplies,None,Durable Medical Equipment & Medical Supplies
2,1881697092,"RHS, INC.",None,None,None,None,1330 TROTWOOD AVE,None,COLUMBIA,TN,...,0.993028,0.999619,1.0,0.993669,1881697092,332B00000X,332B00000X,Durable Medical Equipment & Medical Supplies,None,Durable Medical Equipment & Medical Supplies
3,1326041534,MURFREESBORO CONVALESCENT SERVICE,None,None,None,None,4428 LASCASSAS PIKE,None,MURFREESBORO,TN,...,1.000000,1.000000,1.0,1.000000,1326041534,341600000X,341600000X,Ambulance,None,Ambulance
4,1326041534,MURFREESBORO CONVALESCENT SERVICE,None,None,None,None,4428 LASCASSAS PIKE,None,MURFREESBORO,TN,...,1.000000,1.000000,1.0,1.000000,1326041534,341600000X,341600000X,Ambulance,None,Ambulance


In [2]:
#Performs a SQL query to pull providers located outside of the Nashville CBSA who are referring to hospital
# inside the Nashville CBSA.
# Other CBSAs include:
# cbsa = 17300 Clarksville, TN
# cbsa = 14540 Bowling Green Ky
# cbsa = 27180 Jackson, TN
# cbsa = 18260 Cookeville, TN
# cbsa = 30280 Lewisburg, TN
# cbsa = 43180 Shelbyville, TN
# cbsa = 46100 Tullahoma-Manchester, TN
# cbsa = 32660 McMinnville, TN
# cbsa = 16860 Chattanooga, TN
#WARNING This SQL query takes >2 hours to run. Prevent your PC from sleeping and wait.
query = '''
WITH from_npi AS (
SELECT npi AS from_npi,
    "provider_organization_name_(legal_business_name)" AS from_org_name,
    "provider_last_name_(legal_name)" AS from_last_name,
    provider_first_name AS from_first_name,
    provider_middle_name AS from_middle_name,
    provider_credential_text AS from_credential,
    provider_first_line_business_practice_location_address AS from_address,
    provider_second_line_business_practice_location_address AS from_2_address,
    provider_business_practice_location_address_city_name AS from_city,
    provider_business_practice_location_address_state_name AS from_state,
    substring(nppes.provider_business_practice_location_address_postal_code, 1, 5) AS from_zip
FROM nppes
WHERE entity_type_code = 1.0
), 
surrounding_cbsa AS(
SELECT CAST(zip as TEXT) as zip,
cbsa,
usps_zip_pref_city,
usps_zip_pref_state
FROM cbsa
WHERE cbsa = 17300 
OR cbsa = 14540 
OR cbsa = 27180 
OR cbsa = 18260 
OR cbsa = 30280 
OR cbsa = 43180 
OR cbsa = 46100 
OR cbsa = 32660 
OR cbsa = 16860
),
taxon AS (
SELECT npi, taxonomy_code
FROM taxonomy
WHERE taxonomy_switch = 'Y'
),
codes AS(
SELECT code, classification, specialization, display_name
FROM nucc
)
SELECT *
FROM from_npi
JOIN surrounding_cbsa
on from_npi.from_zip = zip
JOIN taxon
ON from_npi.from_npi = npi
JOIN codes
ON codes.code = taxon.taxonomy_code
'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    from_npi = pd.read_sql(query,db)

db.close()
from_npi.to_csv('data/from_npi.csv')
from_npi.head()

,from_npi,from_org_name,from_last_name,from_first_name,from_middle_name,from_credential,from_address,from_2_address,from_city,from_state,...,res_ratio,bus_ratio,oth_ratio,tot_ratio,npi,taxonomy_code,code,classification,specialization,display_name
0,1003112277,None,SMITH,TABITHA,M,F.N.P.-C,89 MEARS DRIVE,None,WOODBURY,TN,...,0.009001,0.008565,0.0,0.008846,1003112277,363LF0000X,363LF0000X,Nurse Practitioner,Family,Family Nurse Practitioner
1,1003854258,None,YOUNG,KEITH,None,M.D.,205 S MCCRARY ST,None,WOODBURY,TN,...,0.009001,0.008565,0.0,0.008846,1003854258,207Q00000X,207Q00000X,Family Medicine,None,Family Medicine Physician
2,1013238856,None,PINKSTON,GAVIN,BUTLER,MD,205 S MCCRARY ST,None,WOODBURY,TN,...,0.009001,0.008565,0.0,0.008846,1013238856,207Q00000X,207Q00000X,Family Medicine,None,Family Medicine Physician
3,1063059772,None,APPLEGATE,MICAH,KATHERINE,None,119 W HIGH ST,None,WOODBURY,TN,...,0.009001,0.008565,0.0,0.008846,1063059772,225200000X,225200000X,Physical Therapy Assistant,None,Physical Therapy Assistant
4,1073583449,None,COOMES,BERNARD,JOHN,DC,313 W MAIN ST,None,WOODBURY,TN,...,0.009001,0.008565,0.0,0.008846,1073583449,111NI0900X,111NI0900X,Chiropractor,Internist,Internist Chiropractor


In [13]:
#filter hop_team for entity_type 1.0 or 2.0 and transaction_count>=50
#and average_day_wait<50

#Update code later. Don't need npi in the cte's
query = '''
WITH from_npi_list AS (
    SELECT npi, 
    substring(provider_business_practice_location_address_postal_code, 1, 5) AS from_zip
    FROM nppes
    INNER JOIN cbsa
    ON cbsa.zip = from_zip
    WHERE entity_type_code = 1.0 AND cbsa = 17300 
        OR cbsa = 14540 
        OR cbsa = 27180 
        OR cbsa = 18260 
        OR cbsa = 30280 
        OR cbsa = 43180 
        OR cbsa = 46100 
        OR cbsa = 32660 
        OR cbsa = 16860
),
to_npi_list AS (
    SELECT npi,
    substring(provider_business_practice_location_address_postal_code, 1, 5) AS to_zip
    FROM nppes   
    INNER JOIN cbsa
    ON cbsa.zip = to_zip
    WHERE entity_type_code = 2.0 AND cbsa = 34980
)
SELECT *
FROM hop_team
INNER JOIN from_npi_list
ON hop_team.from_npi = from_npi_list.npi
INNER JOIN to_npi_list
ON hop_team.to_npi = to_npi_list.npi
WHERE transaction_count >= 50 
    AND average_day_wait < 50
'''

with sqlite3.connect('data/hop_team.sqlite') as db:
    hop_team = pd.read_sql(query,db)
db.close()
hop_team.to_csv('data/hop_team.csv')
hop_team.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,npi,from_zip,npi,to_zip
0,1518961176,1245233220,264,451,0.929,10.235,1518961176,37421,1245233220,37115
1,1518961176,1245233220,264,451,0.929,10.235,1518961176,37421,1245233220,37115
2,1518961176,1245233220,264,451,0.929,10.235,1518961176,37421,1245233220,37115
3,1518961176,1245233220,264,451,0.929,10.235,1518961176,37421,1245233220,37115
4,1518961176,1245233220,264,451,0.929,10.235,1518961176,37421,1245233220,37115


Clean up columns
    Drop code or taxonomy_code,
    drop one of the Zip code fields. 
    drop npi as it's captured in to_npi
Merge hop_team with to_npi and from_npi
    clean up columns 


In [2]:
#Run this if you have already performed the SQL queries. It's much faster. Also removes unnecessary columns.  
to_npi = pd.read_csv('data/to_npi.csv')
to_npi = to_npi.drop(columns= ['to_last_name',
                               'to_first_name',
                               'to_middle_name',
                               'to_credential',
                               'zip', 'npi',
                               'code', 'taxonomy_code',
                               'res_ratio',
                               'bus_ratio',
                               'oth_ratio',
                               'tot_ratio'])
from_npi = pd.read_csv('data/from_npi.csv')
from_npi = from_npi.drop(columns = ['Unnamed: 0','from_org_name',
                                    'zip',
                                    'npi',
                                    'code', 'taxonomy_code','res_ratio', 'bus_ratio', 'oth_ratio',
       'tot_ratio'])
hop_team = pd.read_csv('data/hop_team.csv')
hop_team = hop_team.drop(columns = ['Unnamed: 0','npi','npi.1'])
hop_team = hop_team.drop_duplicates()


Merge from_npi to_npi and hop_team at this point. 
Further filters: Find Vanderbilt's NPI and filter for to_npi that are not vanderbilt's. 


In [5]:

from_hop_team = pd.merge(left = from_npi, right = hop_team, how = 'inner', on = 'from_npi')
clean_hop_team = pd.merge(left = to_npi, right = from_hop_team, how = 'inner', on = 'to_npi')
clean_hop_team = clean_hop_team.drop(columns = ['Unnamed: 0','from_zip_y',
       'to_zip_y'])
clean_hop_team = clean_hop_team.drop_duplicates()
columns_rename = {'to_zip_x': 'to_zip',
 'cbsa_x': 'to_cbsa' ,
 'usps_zip_pref_city_x': 'to_city',
 'usps_zip_pref_state_x': 'to_state',
 'classification_x': 'to_classification',
 'specialization_x': 'to_specialization',
 'display_name_x':'to_display_name',
 'from_zip_x': 'from_zip',
 'cbsa_y':'from_cbsa',
 'usps_zip_pref_city_y': 'from_city',
 'usps_zip_pref_state_y': 'from_state',
 'classification_y': 'from_classification',
 'specialization_y':'from_specialization',
 'display_name_y': 'from_display_name'}
clean_hop_team = clean_hop_team.rename(columns = columns_rename)
clean_hop_team = clean_hop_team.astype({'to_npi':'object', 'to_zip':'object',
                       'to_cbsa':'object', 'from_npi':'object',
                       'from_zip':'object', 'from_cbsa':'object'})

clean_hop_team.head()

,to_npi,to_org_name,to_address,to_2_address,to_city,to_state,to_zip_x,cbsa_x,usps_zip_pref_city_x,usps_zip_pref_state_x,...,cbsa_y,usps_zip_pref_city_y,usps_zip_pref_state_y,classification_y,specialization_y,display_name_y,patient_count,transaction_count,average_day_wait,std_day_wait
0,1245233220,"NORTHRIDGE SURGERY CENTER, LP",647 MYATT DR,NaN,MADISON,TN,37115,34980,MADISON,TN,...,16860,CHATTANOOGA,TN,Optometrist,NaN,Optometrist,159,160,32.569,28.364
3,1245233220,"NORTHRIDGE SURGERY CENTER, LP",647 MYATT DR,NaN,MADISON,TN,37115,34980,MADISON,TN,...,16860,CHATTANOOGA,TN,Ophthalmology,NaN,Ophthalmology Physician,232,409,0.599,6.955
18,1609879956,"CARIS HEALTHCARE, LP",2525 PERIMETER PLACE DR,SUITE 131,NASHVILLE,TN,37214,34980,NASHVILLE,TN,...,17300,ASHLAND CITY,TN,Family Medicine,NaN,Family Medicine Physician,22,58,1.517,10.659
21,1609879956,"CARIS HEALTHCARE, LP",2525 PERIMETER PLACE DR,SUITE 131,NASHVILLE,TN,37214,34980,NASHVILLE,TN,...,30280,SPRING HILL,TN,Internal Medicine,NaN,Internal Medicine Physician,29,76,0.276,1.845
36,1568464873,DICKSON MEDICAL ASSOCIATES PC,113 HIGHWAY 70 E,NaN,DICKSON,TN,37055,34980,DICKSON,TN,...,17300,ERIN,TN,Family Medicine,NaN,Family Medicine Physician,123,173,27.179,39.565


In [17]:
clean_hop_team.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11707 entries, 0 to 105294
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   to_npi               11707 non-null  object 
 1   to_org_name          11707 non-null  object 
 2   to_address           11707 non-null  object 
 3   to_2_address         3868 non-null   object 
 4   to_city              11707 non-null  object 
 5   to_state             11707 non-null  object 
 6   to_zip               11707 non-null  object 
 7   to_cbsa              11707 non-null  object 
 8   to_city              11707 non-null  object 
 9   to_state             11707 non-null  object 
 10  to_classification    11707 non-null  object 
 11  to_specialization    5001 non-null   object 
 12  to_display_name      11707 non-null  object 
 13  from_npi             11707 non-null  object 
 14  from_last_name       11699 non-null  object 
 15  from_first_name      11707 non-null

In [15]:
clean_hop_team.nunique()

to_npi                  614
to_org_name             522
to_address              497
to_2_address            112
to_city                  41
to_state                  2
to_zip                   62
to_cbsa                   1
to_city                  39
to_state                  1
to_classification        58
to_specialization        63
to_display_name         119
from_npi               2393
from_last_name         1713
from_first_name         945
from_middle_name        612
from_credential         185
from_address           1067
from_2_address          264
from_city                96
from_state                5
from_zip                108
from_cbsa                 9
from_city                89
from_state                3
from_classification      43
from_specialization      68
from_display_name       116
patient_count           753
transaction_count      1127
average_day_wait       8309
std_day_wait           8636
dtype: int64

Vanderbilt NPI 1396882205

calculate some summary statistics. What specialties are referring to Nashville, What specialities are not. Where are the specialties referring to in Nashville. 
Still need to geocode the columns for a map. but I don't think it would really have much value honestly. 

In [14]:
clean_hop_team.to_csv('data/clean_hop_team.csv')

put the clean_hop_team.csv in neo4J and get the community group. push back out to CSV and analyze.